In [29]:
import pandas as pd

df = pd.read_csv('../../dataset/cleaned/demographic.csv', header=1, index_col=0)

C:\Users\brema\AppData\Local\Temp\ipykernel_20768\3816664273.py:3: DtypeWarning: Columns (2,10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../../dataset/cleaned/demographic.csv', header=1, index_col=0)


In [45]:
from geopy.geocoders import Nominatim
import matplotlib.pyplot as plt
plt.rcParams['font.family']='Tahoma'
plt.rcParams['font.size']=13

def extract_indexes(df):
    province_names = df['ชื่อจังหวัด'].unique()
    registry_names = df['ชื่อสำนักทะเบียน'].unique()
    district_names = df['ชื่อตำบล'].unique()
    return province_names, registry_names, district_names

bangkok_df = df[df['ชื่อจังหวัด']=='กรุงเทพมหานคร']
province_names, registry_names, district_names = extract_indexes(bangkok_df)

locations = []

geolocator = Nominatim(user_agent="real_deal")

registry_df = bangkok_df[bangkok_df['ปีเดือน'].isin(map(str, list(range(6601, 6608))))].groupby(['ชื่อสำนักทะเบียน']).sum()[['ผลรวมประชากรทั้งหมด']].reset_index().sort_values(by=['ผลรวมประชากรทั้งหมด'])
registry_names = registry_df['ชื่อสำนักทะเบียน'].unique()
print(registry_names)

for registry in registry_names:
    location = geolocator.geocode(registry)
    locations.append({"lat": location.latitude, "lon": location.longitude, "name": registry})

['เขตสัมพันธวงศ์' 'เขตพระนคร' 'เขตป้อมปราบศัตรูพ่าย' 'เขตปทุมวัน'
 'เขตบางรัก' 'เขตบางกอกใหญ่' 'เขตพญาไท' 'เขตราชเทวี' 'เขตคลองสาน'
 'เขตสาทร' 'เขตยานนาวา' 'เขตราษฎร์บูรณะ' 'เขตดุสิต' 'เขตบางคอแหลม'
 'เขตทวีวัฒนา' 'เขตวัฒนา' 'เขตห้วยขวาง' 'เขตบางนา' 'เขตพระโขนง'
 'เขตบางพลัด' 'เขตคลองเตย' 'เขตคันนายาว' 'เขตสะพานสูง' 'เขตธนบุรี'
 'เขตบางกอกน้อย' 'เขตหลักสี่' 'เขตบางบอน' 'เขตตลิ่งชัน' 'เขตวังทองหลาง'
 'เขตดินแดง' 'เขตลาดพร้าว' 'เขตบางซื่อ' 'เขตภาษีเจริญ' 'เขตสวนหลวง'
 'เขตทุ่งครุ' 'เขตบึงกุ่ม' 'เขตมีนบุรี' 'เขตบางกะปิ' 'เขตจอมทอง'
 'เขตจตุจักร' 'เขตหนองแขม' 'เขตดอนเมือง' 'เขตลาดกระบัง' 'เขตหนองจอก'
 'เขตประเวศ' 'เขตบางขุนเทียน' 'เขตบางเขน' 'เขตบางแค' 'เขตสายไหม'
 'เขตคลองสามวา']


In [52]:
from geopy.geocoders import Nominatim
import folium
import matplotlib.pyplot as plt
plt.rcParams['font.family']='Tahoma'
plt.rcParams['font.size']=13

def colfunc(val, minval, maxval, startcolor, stopcolor):
    """ Convert value in the range minval...maxval to a color in the range
        startcolor to stopcolor. The colors passed and the one returned are
        composed of a sequence of N component values (e.g. RGB).
    """
    f = float(val-minval) / (maxval-minval)
    return tuple(f*(b-a)+a for (a, b) in zip(startcolor, stopcolor))

def rgb_to_hex(r, g, b):
    return '#{:02x}{:02x}{:02x}'.format(r, g, b)

RED, YELLOW, GREEN  = (255, 0, 0), (255, 255, 0), (0, 255, 0)
CYAN, BLUE, MAGENTA = (0, 255, 255), (0, 0, 255), (255, 0, 255)
steps = len(locations)+1
minval, maxval = 0.0, 255.0
incr = (maxval-minval)/steps

geolocator = Nominatim(user_agent="real_deal")

location = geolocator.geocode("เขตสัมพันธวงศ์")
m = folium.Map(location=[location.latitude, location.longitude], zoom_start=12)

for i, location in enumerate(locations):
    val = minval + round(i*incr, 1)
    r, g, b = colfunc(val, minval, maxval, BLUE, MAGENTA)
    hex_color = rgb_to_hex(int(r),int(g),int(b))
    folium.Circle(radius=400, location=[location['lat'], location['lon']], popup=location['name'], color=hex_color, fill=True, fill_color=hex_color).add_to(m)
m